In [1]:
from models.VAE import CVAE
from experiments import MNISTGenExperiment
from lightning.pytorch import Trainer
from lightning.pytorch.utilities.seed import seed_everything
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from pathlib import Path
import os

DATA_PATH = os.path.join(os.getcwd(),"data\\MNIST_data")
OUTPUT_DIR = os.path.join(os.path.join(os.path.join(os.getcwd(), "runs"), "MNISTGen"), "logs")
NAME = "CVAE"


seed_everything(1234, True)

vae_model = CVAE((1, 28, 28), 8)
experiment = MNISTGenExperiment.MNISTGenExperiment(vae_model)

data = MNISTGenExperiment.get_data(DATA_PATH)

Path(OUTPUT_DIR).mkdir(exist_ok=True, parents=True)

tb_logger =  TensorBoardLogger(save_dir=OUTPUT_DIR,
                               name=NAME)

runner = Trainer(logger=tb_logger,
                 callbacks=[
                     LearningRateMonitor(),
                     ModelCheckpoint(save_top_k=2, 
                                     dirpath =os.path.join(tb_logger.log_dir , "checkpoints"), 
                                     monitor= "val_loss",
                                     save_last= True),
                 ],
                 max_epochs=20)


runner.fit(experiment, datamodule=data)


c:\Users\lotan\AppData\Local\Programs\Python\Python37\lib\site-packages\lightning\pytorch\utilities\seed.py:48: LightningDeprecationWarning: `lightning.pytorch.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning.fabric.utilities.seed.seed_everything` instead.
  "`lightning.pytorch.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be"
Global seed set to 1234
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type | Params
-------------------------------
0 | model | CVAE | 283 K 
-------------------------------
283 K     Trainable params
0         Non-trainable params
283 K     Total params
1.133     Total estimated model params size (MB)


10000 0 10000
10000 0 10000


Sanity Checking: 0it [00:00, ?it/s]

a
b


c:\Users\lotan\AppData\Local\Programs\Python\Python37\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:229: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'torch.utils.data.dataset.Subset'>